__Assignment 7__

1. [Import](#Import)
1. [Assignment 7](#Assignment-7)
    1. [Load data](#load-data)
    1. [Train default model](#Train-default-model)    
    1. [Optimize model parameters](#Optimize-model-parameters)    
    1. [Optimize model parameters on test set only (do not do this)](#Optimize-model-parameters-on-test-set-only-(do-not-do-this))    

# Import

<a id = 'Import'></a>

In [6]:
# standard libary and settings
import os
import sys
import warnings

warnings.simplefilter("ignore")

# data extensions and settings
import numpy as np

np.set_printoptions(threshold=np.inf, suppress=True)
import pandas as pd

pd.set_option("display.max_rows", 500)
pd.options.display.float_format = "{:,.6f}".format

# modeling extensions
from sklearn.metrics import precision_score, recall_score, f1_score, explained_variance_score, mean_squared_log_error, mean_absolute_error, median_absolute_error, mean_squared_error, r2_score, confusion_matrix, roc_curve, accuracy_score, roc_auc_score, homogeneity_score, completeness_score, classification_report, silhouette_samples
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures, OrdinalEncoder, LabelEncoder, OneHotEncoder, KBinsDiscretizer, QuantileTransformer, PowerTransformer, MinMaxScaler
from sklearn.svm import SVC, SVR

# Assignment 7

Assignment Content:

1. Implement an SVM classifier using Jupyter Notebook and Python as well as the sklearn package using any dataset of your choosing 
2. Implement cross-validation and train your classifier on the training data, then test the results on the test data. 
3. Tweak the kernel functions and regularization parameters as to perform well both on the training and test data. 
4. Finally, tweak the kernel functions and regularization parameters as to perform the best on the test data (meaning find the optimal test accuracy, not caring too much about the training accuracy). Report on your findings, printing them out to the console in Jupyter Notebook.


<a id = 'Assignment-7'></a>

## Load data

<a id = 'load-data'></a>

In [7]:
# load and inspect data
df_wine = pd.read_csv("s3://tdp-ml-datasets/misc/wine.data", header=None)

df_wine.columns = [
    "Class label",
    "Alcohol",
    "Malic acid",
    "Ash",
    "Alcalinity of ash",
    "Magnesium",
    "Total phenols",
    "Flavanoids",
    "Nonflavanoid phenols",
    "Proanthocyanins",
    "Color intensity",
    "Hue",
    "OD280/OD315 of diluted wines",
    "Proline",
]
df_wine[:5]

Class label   Alcohol  Malic acid      Ash  Alcalinity of ash  Magnesium  \
0            1 14.230000    1.710000 2.430000          15.600000        127   
1            1 13.200000    1.780000 2.140000          11.200000        100   
2            1 13.160000    2.360000 2.670000          18.600000        101   
3            1 14.370000    1.950000 2.500000          16.800000        113   
4            1 13.240000    2.590000 2.870000          21.000000        118   

   Total phenols  Flavanoids  Nonflavanoid phenols  Proanthocyanins  \
0       2.800000    3.060000              0.280000         2.290000   
1       2.650000    2.760000              0.260000         1.280000   
2       2.800000    3.240000              0.300000         2.810000   
3       3.850000    3.490000              0.240000         2.180000   
4       2.800000    2.690000              0.390000         1.820000   

   Color intensity      Hue  OD280/OD315 of diluted wines  Proline  
0         5.640000 1.040000                      3.920000     1065  
1         4.380000 1.050000                      3.400000     1050  
2         5.680000 1.030000                      3.170000     1185  
3         7.800000 0.860000                      3.450000     1480  
4         4.320000 1.040000                      2.930000      735

In [8]:
# return counts for each class label
np.unique(df_wine["Class label"].values, return_counts=True)

(array([1, 2, 3]), array([59, 71, 48]))

In [9]:
# drop class 1 so that we have a binary classification problem
df_wine = df_wine[df_wine["Class label"] != 1]

In [10]:
# split labels and features
y = df_wine["Class label"].values
X = df_wine.iloc[:, 1:].values

# encode labels
le = LabelEncoder()
y = le.fit_transform(y)

# split into train/test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=1, stratify=y
)

## Train default model

<a id = 'Train-default-model'></a>

In [11]:
# cross validation on training data
pipe = make_pipeline(StandardScaler(), SVC(random_state=10))
scores = cross_val_score(
    pipe, X_train, y_train, scoring="accuracy", cv=10
)
print(
    "CV accuracy on training data: {:.3f} +/- {:.3f}".format(
        np.mean(scores), np.std(scores)
    )
)

CV accuracy on training data: 0.980 +/- 0.060


In [12]:
# fit best model, create predictions and review accuracy
model = pipe.fit(X_train, y_train)
y_preds_test = model.predict(X_test)
print(
    "Accuracy: {:.3f}".format(accuracy_score(y_true=y_test, y_pred=y_preds_test))
)

Accuracy: 0.983


> Remarks - The default SVC algorithm uses a C of 1.0 and an RBF kernel.

## Optimize model parameters

<a id = 'Optimize-model-parameters'></a>

In [13]:
# use GridSearchCV to perform CV over several different combination of parameters
pipe = make_pipeline(StandardScaler(), SVC(random_state=1))
param_range = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [
    {"svc__C": param_range, "svc__kernel": ["linear"]},
    {"svc__C": param_range, "svc__gamma": param_range, "svc__kernel": ["rbf"]},
]
gs = GridSearchCV(
    estimator=pipe, param_grid=param_grid, scoring="accuracy", cv=10, n_jobs=-1
)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.9830508474576272
{'svc__C': 0.01, 'svc__kernel': 'linear'}


In [14]:
# fit best model, create predictions and review accuracy
model = gs.fit(X_train, y_train)
y_preds_test = model.predict(X_test)
print(
    "Accuracy: {:.3f}".format(accuracy_score(y_true=y_test, y_pred=y_preds_test))
)

Accuracy: 0.983


> Remarks - GridSearchCV performs slightly better on the training data and similarly on the test data compared to the default SVC algorithm but instead chooses a C of 0.01 and a linear kernel. Between the two models, this would be the better choice as it is the simpler model of the two.

## Optimize model parameters on test set only (do not do this)

<a id = 'Optimize-model-parameters-test-set-only-(do-not-do-this)'></a>

In [15]:
# use GridSearchCV to perform CV over several different combination of parameters
pipe = make_pipeline(StandardScaler(), SVC(random_state=1))
param_range = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [
    {"svc__C": param_range, "svc__kernel": ["linear"]},
    {"svc__C": param_range, "svc__gamma": param_range, "svc__kernel": ["rbf"]},
]
gs = GridSearchCV(
    estimator=pipe, param_grid=param_grid, scoring="accuracy", cv=10, n_jobs=-1
)
gs = gs.fit(X_test, y_test)
print(gs.best_score_)
print(gs.best_params_)

1.0
{'svc__C': 10.0, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}


> Remarks - On the test set, GridSearchCV chose a very different model than it did for the training set. In this case, the ideal parameters were C = 10.0, gamma = 0.01 and an RBF kernel.